# Folpetti - Predição Jogos da Copa do Mundo de Futebol

Aplicação de Machine Learning para predição do resultado dos jogos em 2022

**CR**oss **I**ndustry **S**tandard **P**rocess for **D**ata **M**ining (CRISP-DM) 

<img width="300px" style="text-align:center;" src="https://upload.wikimedia.org/wikipedia/commons/b/b9/CRISP-DM_Process_Diagram.png" />


## Objetivo (Entendendo do Negócio)

Devemos gerar o resultado dos jogos com a quantidade de gols para os dois times, acreditando que não faz sentido gerar a predição de dois valores indepentendes vamos adotar a classificação de eventos

> **Classificação**: Aplicação supervisionada de Machine Learning para encaixar o registro em um grupo, dentre os pré-definidos


Com as seguintes regras:

- Não pode ser usado nenhum tipo de enriquecimento, importação de dados externos ou introdução manual de dados;
- Replicabilidade é essencial, como é uma atividade que deve ser validada no seu processo e aplicabilidade, os valores aletórios devem ser controlados para sempre ter os mesmos resultados.

## Coleta, Compreensão e Preparação

Usando somentes base dados presentes no repositório [Eduardo Ruela - Paul Octopus 2022](https://github.com/edruela/paul-octopus-2022)

In [ ]:
# Importação da biblioteca para lidar com os dados
import pandas as pd
# Importação de biblioteca matemática
import numpy as np

# Definição do caminho padrão das informações
path_source = 'https://raw.githubusercontent.com/edruela/paul-octopus-2022/main/'

# Definição de função para rodar por linha de comando
# display = print

### Remover os outliers dos valores 

Usando o critério do 150% do IQR (Q3 - Q1) 

In [ ]:
# Definição da função para remover outliers
def remove_outlier(df_in, col_name):
    # Pegar limite primeiro e terceiro quadrante
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)

    # Alcance do primeiro e terceiro quadrante
    iqr = q3 - q1

    # Definição de limites 
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    # Realizar a filtragem dos outliers
    list_remove = (df_in[col_name] < limite_inferior) | (df_in[col_name] > limite_superior)
    print(f"Campo {col_name} limites {limite_inferior:.3f} a {limite_superior:.3f} contagem {len(df_in[list_remove].index)}")

    return df_in[~list_remove]

### Histórico de resultados (historical-results.csv)

Uma listagem dos resultados dos jogos de outros anos com dados

In [ ]:
# Leitura do "Histórico de resultados"
df_results = pd.read_csv(path_source + 'historical-results.csv')

# Visualização dos dados importados
df_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


Informações da fonte de dados

In [ ]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44060 entries, 0 to 44059
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        44060 non-null  object 
 1   home_team   44060 non-null  object 
 2   away_team   44060 non-null  object 
 3   home_score  44059 non-null  float64
 4   away_score  44059 non-null  float64
 5   tournament  44060 non-null  object 
 6   city        44060 non-null  object 
 7   country     44060 non-null  object 
 8   neutral     44060 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.7+ MB


In [ ]:
# Remover valores nulos na coluna de home score
df_results = df_results[df_results.home_score.notna()]

# Remover outliers da pontuação do time da casa
df_results = remove_outlier(df_results, 'home_score')
# Remover outliers da pontuação do time da visitante
df_results = remove_outlier(df_results, 'away_score')

print('\n')
df_results.info()

Campo home_score limites -0.500 a 3.500 contagem 5520
Campo away_score limites -3.000 a 5.000 contagem 589


<class 'pandas.core.frame.DataFrame'>
Int64Index: 37950 entries, 0 to 44058
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        37950 non-null  object 
 1   home_team   37950 non-null  object 
 2   away_team   37950 non-null  object 
 3   home_score  37950 non-null  float64
 4   away_score  37950 non-null  float64
 5   tournament  37950 non-null  object 
 6   city        37950 non-null  object 
 7   country     37950 non-null  object 
 8   neutral     37950 non-null  bool   
dtypes: bool(1), float64(2), object(6)
memory usage: 2.6+ MB


Para poder absorver as informações de data convertemos em formato de data e separamos as informações de ano, mês, dia da semana e semana do ano

In [ ]:
# Converter texto para date em pandas
df_results['datetime'] = pd.to_datetime(df_results.date, format='%Y-%m-%d')

# Extrair trechos da data
def get_dates(datasource):
  datasource['dayofweek'] = pd.DatetimeIndex(datasource.datetime).dayofweek.astype(str)
  datasource['weekofyear'] = datasource.datetime.dt.isocalendar().week
  datasource['month'] = pd.DatetimeIndex(datasource.datetime).month
  datasource['year'] = pd.DatetimeIndex(datasource.datetime).year

  return datasource

# Aplicação da função
get_dates(df_results)

# Visualizar os campos
df_results[[
      'date', 
      'datetime', 
      'dayofweek', 
      'weekofyear', 
      'month', 
      'year'
    ]].head()

,date,datetime,dayofweek,weekofyear,month,year
0,1872-11-30,1872-11-30,5,48,11,1872
2,1874-03-07,1874-03-07,5,10,3,1874
3,1875-03-06,1875-03-06,5,9,3,1875
4,1876-03-04,1876-03-04,5,9,3,1876
6,1877-03-03,1877-03-03,5,9,3,1877


Gerar fonte de dados com os nomes dos países para comparar com os outros arquivos

In [ ]:
# Juntas as duas listas
results_countries = df_results.home_team.to_list() + df_results.away_team.to_list()
# Transformar a lista em DataFrame
df_results_countries = pd.DataFrame(results_countries, columns = ['country']).sort_values('country')
# Separar os valores únicos 
results_countries = df_results_countries.country.unique()

# Transformar a lista única em DataFrame
df_results_countries = pd.DataFrame(results_countries, columns = ['country']).sort_values('country')
df_results_countries

,country
0,Abkhazia
1,Afghanistan
2,Albania
3,Alderney
4,Algeria
...,...
306,Yugoslavia
307,Zambia
308,Zanzibar
309,Zimbabwe


Vamos tirar umas métricas desses resultados como porcentagem de vitórias e empates

In [ ]:
# Apurar o vencedor das disputas
def getwinner(row):
  if row.home_score > row.away_score: 
    return row.home_team 
  if row.home_score < row.away_score: 
    return row.away_team
  return 'draw'

# Selecionar o vencedor do evento
df_results['id'] = df_results.index
df_results['winner'] =  df_results.apply(getwinner, axis=1)

# Totalizar o número de vitórias por seleção
winner_count = df_results.groupby('winner').id.count().rename("count_winner").reset_index(name='count_winner').rename(columns={"winner":"country"})
winner_count.info()
print('\n')

# Totalizar o número de vitórias por seleção em jogos da Copa
fifa_winner_count = df_results[df_results.tournament == "FIFA World Cup"].groupby('winner').id.count().rename("count_fifa_winner").reset_index(name='count_fifa_winner').rename(columns={"winner":"country"})
fifa_winner_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country       300 non-null    object
 1   count_winner  300 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.8+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   country            63 non-null     object
 1   count_fifa_winner  63 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


In [ ]:
# Contagem da presença das seleções
home_count = df_results.groupby('home_team').id.count().rename("home_count").reset_index(name="home_count").rename(columns={"home_team":"country"})
away_count = df_results.groupby('away_team').id.count().rename("away_count").reset_index(name="away_count").rename(columns={"away_team":"country"})

# Contagem da presença das seleções nas Copas do Mundo
home_count_fifa = df_results[df_results.tournament == "FIFA World Cup"].groupby('home_team').id.count().rename("home_count_fifa").reset_index(name="home_count_fifa").rename(columns={"home_team":"country"})
away_count_fifa = df_results[df_results.tournament == "FIFA World Cup"].groupby('away_team').id.count().rename("away_count_fifa").reset_index(name="away_count_fifa").rename(columns={"away_team":"country"})

# Juntar a contagem de participaão em jogos e jogos da Copa
country_summary = pd.merge(home_count, away_count, how='outer',
                           left_on=['country'],
                           right_on=['country'])
country_summary = pd.merge(country_summary, home_count_fifa, how='left',
                           left_on=['country'],
                           right_on=['country'])
country_summary = pd.merge(country_summary, away_count_fifa, how='left',
                           left_on=['country'],
                           right_on=['country'])

# Preencher as lacunas com zero e totalizar os valores 
country_summary.loc[country_summary.home_count.isna(), ['home_count']] = 0
country_summary.loc[country_summary.away_count.isna(), ['away_count']] = 0
country_summary['total'] = country_summary.home_count + country_summary.away_count

# Preencher as lacunas com zero com resultados FIFA e totalizar os valores
country_summary.loc[country_summary.home_count_fifa.isna(), ['home_count_fifa']] = 0
country_summary.loc[country_summary.away_count_fifa.isna(), ['away_count_fifa']] = 0
country_summary['total_fifa'] = country_summary.home_count_fifa + country_summary.away_count_fifa

country_summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 0 to 310
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          311 non-null    object 
 1   home_count       311 non-null    float64
 2   away_count       311 non-null    float64
 3   home_count_fifa  311 non-null    float64
 4   away_count_fifa  311 non-null    float64
 5   total            311 non-null    float64
 6   total_fifa       311 non-null    float64
dtypes: float64(6), object(1)
memory usage: 19.4+ KB


In [ ]:
# Calcular a mediana da pontuação por países
home_score_by_country = df_results[['home_team', 'home_score']].rename(columns={'home_team': 'country', 'home_score': 'score'})
away_score_by_country = df_results[['away_team', 'away_score']].rename(columns={'away_team': 'country', 'away_score': 'score'})

scores_by_country = pd.concat([home_score_by_country, away_score_by_country]).groupby('country').score.median().rename('median_score')
display(scores_by_country.head())

# Calcular a mediana da pontuação por países na Copa do Mundo
home_score_fifa = df_results[df_results.tournament == "FIFA World Cup"][['home_team', 'home_score']].rename(columns={'home_team': 'country', 'home_score': 'score'})
away_score_fifa = df_results[df_results.tournament == "FIFA World Cup"][['away_team', 'away_score']].rename(columns={'away_team': 'country', 'away_score': 'score'})

scores_by_country_fifa = pd.concat([home_score_fifa, away_score_fifa]).groupby('country').score.median().rename('median_fifa_score')
display(scores_by_country_fifa.head())

# Juntar com a mediana das pontuações 
country_summary = pd.merge(country_summary, scores_by_country, how='left', left_on=['country'], right_on=['country'])
country_summary = pd.merge(country_summary, scores_by_country_fifa, how='left', left_on=['country'], right_on=['country'])

# Preencher as lacunas com zero
country_summary.loc[country_summary.median_score.isna(), ['median_score']] = 0
country_summary.loc[country_summary.median_fifa_score.isna(), ['median_fifa_score']] = 0

country_summary.head()

country
Abkhazia       1.0
Afghanistan    1.0
Albania        1.0
Alderney       1.0
Algeria        1.0
Name: median_score, dtype: float64

country
Algeria      1.0
Angola       0.0
Argentina    1.0
Australia    1.0
Austria      1.0
Name: median_fifa_score, dtype: float64

,country,home_count,away_count,home_count_fifa,away_count_fifa,total,total_fifa,median_score,median_fifa_score
0,Abkhazia,16.0,10.0,0.0,0.0,26.0,0.0,1.0,0.0
1,Afghanistan,37.0,62.0,0.0,0.0,99.0,0.0,1.0,0.0
2,Albania,182.0,144.0,0.0,0.0,326.0,0.0,1.0,0.0
3,Alderney,6.0,5.0,0.0,0.0,11.0,0.0,1.0,0.0
4,Algeria,268.0,225.0,6.0,7.0,493.0,13.0,1.0,1.0


In [ ]:
#
country_summary = pd.merge(country_summary, winner_count, how='left', left_on=['country'], right_on=['country'])
country_summary.loc[country_summary.count_winner.isna(), ['count_winner']] = 0

#
country_summary = pd.merge(country_summary, fifa_winner_count, how='left', left_on=['country'], right_on=['country'])
country_summary.loc[country_summary.count_fifa_winner.isna(), ['count_fifa_winner']] = 0

# Visualização da fonte de dados nesse momento
print(country_summary.info())
display(country_summary.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 0 to 310
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            311 non-null    object 
 1   home_count         311 non-null    float64
 2   away_count         311 non-null    float64
 3   home_count_fifa    311 non-null    float64
 4   away_count_fifa    311 non-null    float64
 5   total              311 non-null    float64
 6   total_fifa         311 non-null    float64
 7   median_score       311 non-null    float64
 8   median_fifa_score  311 non-null    float64
 9   count_winner       311 non-null    float64
 10  count_fifa_winner  311 non-null    float64
dtypes: float64(10), object(1)
memory usage: 29.2+ KB
None


,country,home_count,away_count,home_count_fifa,away_count_fifa,total,total_fifa,median_score,median_fifa_score,count_winner,count_fifa_winner
0,Abkhazia,16.0,10.0,0.0,0.0,26.0,0.0,1.0,0.0,10.0,0.0
1,Afghanistan,37.0,62.0,0.0,0.0,99.0,0.0,1.0,0.0,28.0,0.0
2,Albania,182.0,144.0,0.0,0.0,326.0,0.0,1.0,0.0,88.0,0.0
3,Alderney,6.0,5.0,0.0,0.0,11.0,0.0,1.0,0.0,3.0,0.0
4,Algeria,268.0,225.0,6.0,7.0,493.0,13.0,1.0,1.0,197.0,3.0


In [ ]:
country_summary['ratio_win'] = country_summary.count_winner / country_summary.total
country_summary['ratio_win_fifa'] = country_summary.count_fifa_winner / country_summary.total_fifa

country_summary.loc[country_summary.ratio_win_fifa.isna(), ['ratio_win_fifa']] = 0
country_summary.loc[country_summary.ratio_win_fifa.isna(), ['ratio_win_fifa']] = 0

country_summary = country_summary[['country', 'ratio_win', 'ratio_win_fifa', 'median_score', 'median_fifa_score']] 

# Visualização da fonte de dados estruturado
print(country_summary.info())
display(country_summary.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 0 to 310
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            311 non-null    object 
 1   ratio_win          311 non-null    float64
 2   ratio_win_fifa     311 non-null    float64
 3   median_score       311 non-null    float64
 4   median_fifa_score  311 non-null    float64
dtypes: float64(4), object(1)
memory usage: 14.6+ KB
None


,country,ratio_win,ratio_win_fifa,median_score,median_fifa_score
0,Abkhazia,0.384615,0.000000,1.0,0.0
1,Afghanistan,0.282828,0.000000,1.0,0.0
2,Albania,0.269939,0.000000,1.0,0.0
3,Alderney,0.272727,0.000000,1.0,0.0
4,Algeria,0.399594,0.230769,1.0,1.0


In [ ]:
# Buscar os valores do home e away teams
home_country_summary = country_summary.rename(columns=
  {'country':'home_team', 'ratio_win':'home_ratio_win', 'ratio_win_fifa':'home_ratio_win_fifa', 'median_score': 'home_median_score', 'median_fifa_score': 'home_median_fifa_score'})
away_country_summary = country_summary.rename(columns=
  {'country':'away_team', 'ratio_win':'away_ratio_win', 'ratio_win_fifa':'away_ratio_win_fifa', 'median_score': 'away_median_score', 'median_fifa_score': 'away_median_fifa_score'})

# Resumir os valores médios
results_summary = country_summary.median(numeric_only=True)
results_summary

ratio_win            0.333333
ratio_win_fifa       0.000000
median_score         1.000000
median_fifa_score    0.000000
dtype: float64

In [ ]:
'''
#
home_score = df_results[['home_team', 'home_score']].rename(columns={'home_team': 'country', 'home_score': 'score'})
away_score = df_results[['away_team', 'away_score']].rename(columns={'away_team': 'country', 'away_score': 'score'})
scores = pd.concat([home_score, away_score])

#
home_score_fifa = df_results[df_results.tournament == 'FIFA World Cup'][['home_team', 'home_score']].rename(columns={'home_team': 'country', 'home_score': 'score'})
away_score_fifa = df_results[df_results.tournament == 'FIFA World Cup'][['away_team', 'away_score']].rename(columns={'away_team': 'country', 'away_score': 'score'})
scores_fifa = pd.concat([home_score_fifa, away_score_fifa])
'''

"\n#\nhome_score = df_results[['home_team', 'home_score']].rename(columns={'home_team': 'country', 'home_score': 'score'})\naway_score = df_results[['away_team', 'away_score']].rename(columns={'away_team': 'country', 'away_score': 'score'})\nscores = pd.concat([home_score, away_score])\n\n#\nhome_score_fifa = df_results[df_results.tournament == 'FIFA World Cup'][['home_team', 'home_score']].rename(columns={'home_team': 'country', 'home_score': 'score'})\naway_score_fifa = df_results[df_results.tournament == 'FIFA World Cup'][['away_team', 'away_score']].rename(columns={'away_team': 'country', 'away_score': 'score'})\nscores_fifa = pd.concat([home_score_fifa, away_score_fifa])\n"

In [ ]:
def get_country_summary(datasource):
  # Unir o "Histórico de resultados" com "Resumo dos resultados"
  result = pd.merge(datasource, home_country_summary, how='left', 
                    left_on=['home_team'], 
                    right_on=['home_team'])
  result = pd.merge(result, away_country_summary, how='left', 
                    left_on=['away_team'], 
                    right_on=['away_team'])

  # Inclusão de valores sumarizados dos resultados
  result['summary_ratio_win'] = results_summary.ratio_win
  result['summary_ratio_win_fifa'] = results_summary.ratio_win_fifa
  result['summary_median_score'] = results_summary.median_score
  result['summary_median_fifa_score'] = results_summary.median_fifa_score

  '''
  # Armazenar percentils para scores geral
  result['summary_min_score'] = scores.min().score
  result['summary_qt1_score'] = scores.quantile(.25).score
  result['summary_qt3_score'] = scores.quantile(.75).score
  result['summary_max_score'] = scores.max().score

  # Armazer percentils para scores na FIFA
  result['summary_min_score_fifa'] = scores_fifa.min().score
  result['summary_qt1_score_fifa'] = scores_fifa.quantile(.25).score
  result['summary_qt3_score_fifa'] = scores_fifa.quantile(.75).score  
  result['summary_max_score_fifa'] = scores_fifa.max().score
  '''

  return result

# Chamada da função de sumarização
df_results = get_country_summary(df_results)

# Preparar a fonte de dados eliminando colunas não usadas
df_results = df_results.drop(columns=['winner', 'id'])

# Visualizar resultado
df_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,datetime,...,home_median_score,home_median_fifa_score,away_ratio_win,away_ratio_win_fifa,away_median_score,away_median_fifa_score,summary_ratio_win,summary_ratio_win_fifa,summary_median_score,summary_median_fifa_score
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1872-11-30,...,1.0,1.0,0.511390,0.415385,1.0,1.0,0.333333,0.0,1.0,0.0
1,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,1874-03-07,...,1.0,1.0,0.511390,0.415385,1.0,1.0,0.333333,0.0,1.0,0.0
2,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1875-03-06,...,1.0,1.0,0.443515,0.150000,1.0,1.0,0.333333,0.0,1.0,0.0
3,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,1876-03-04,...,1.0,1.0,0.511390,0.415385,1.0,1.0,0.333333,0.0,1.0,0.0
4,1877-03-03,England,Scotland,1.0,3.0,Friendly,London,England,False,1877-03-03,...,1.0,1.0,0.443515,0.150000,1.0,1.0,0.333333,0.0,1.0,0.0


### Jogos por resolvidos por pênaltis (shootouts.csv)

Listagem dos jogos resolvidos nos pênaltis com a indicação do vencedor

In [ ]:
# Leitura dos jogos por "Resolvidos por pênaltis"
df_shootouts = pd.read_csv(path_source + 'shootouts.csv')

# Acesso da identificação do index
df_shootouts['id'] = df_shootouts.index

# Visualização dos dados importados
df_shootouts.head()

,date,home_team,away_team,winner,id
0,1967-08-22,India,Taiwan,Taiwan,0
1,1971-11-14,South Korea,Vietnam Republic,South Korea,1
2,1972-05-17,Thailand,South Korea,South Korea,2
3,1972-05-19,Thailand,Cambodia,Thailand,3
4,1973-04-21,Senegal,Ghana,Ghana,4


In [ ]:
# Contagem de vitórias dos times
count_winner = df_shootouts.groupby(['winner']).id.count().rename('count_winner')
# Converter para data frame
country_shootouts = count_winner.reset_index(name='count_winner')
# Renomear coluna para 
country_shootouts = country_shootouts.rename(columns={"winner": "country"})

# Contagem de seleções em home
home_count = df_shootouts.groupby(['home_team']).id.count().rename('home_count')
# Mergiar com os resultados de vitória
country_shootouts = pd.merge(country_shootouts, home_count,
                              how='left',
                              left_on=['country'],
                              right_on=['home_team'])

# Contagem de seleções em away
away_count = df_shootouts.groupby(['away_team']).id.count().rename('away_count')
# Mergiar com os resultados de vitória
country_shootouts = pd.merge(country_shootouts, away_count,
                              how='left',
                              left_on=['country'],
                              right_on=['away_team'])

# Preencher as lacunas com zero
country_shootouts.loc[country_shootouts.home_count.isna(), ['home_count']] = 0
country_shootouts.loc[country_shootouts.away_count.isna(), ['away_count']] = 0

# Calcular a porcentagem de vitórias
country_shootouts['ratio_win_shootouts'] = country_shootouts.apply(lambda row: row.count_winner / (row.home_count + row.away_count), axis=1)

# Preparar a fonte de dados eliminando colunas não usadas
country_shootouts = country_shootouts.drop(columns=['count_winner', 'home_count', 'away_count'])

country_shootouts.head()

,country,ratio_win_shootouts
0,Algeria,0.5
1,Angola,0.7
2,Antigua and Barbuda,1.0
3,Argentina,0.6
4,Aruba,1.0


In [ ]:
def get_count_shootouts(datasource):
  ratio_win_shootouts_home = country_shootouts.rename(columns={"country":"home_team", "ratio_win_shootouts":"home_ratio_win_shootouts"})

  # Unir o "Histórico de resultados" com "Resolvidos por pênaltis"
  result = pd.merge(datasource, ratio_win_shootouts_home, how='left',
                                left_on=['home_team'],
                                right_on=['home_team'])
  
  away_ratio_win_shootouts = country_shootouts.rename(columns={"country": "away_team", "ratio_win_shootouts":"away_ratio_win_shootouts"})

  # Unir o "Histórico de resultados" com "Resolvidos por pênaltis"
  result = pd.merge(result, away_ratio_win_shootouts, how='left',
                                left_on=['away_team'],
                                right_on=['away_team'])
  
  # Completar valores nulos
  result.loc[result.home_ratio_win_shootouts.isna(), ['home_ratio_win_shootouts']] = 0
  result.loc[result.away_ratio_win_shootouts.isna(), ['away_ratio_win_shootouts']] = 0

  return result
  
# Executação da junção e apresentação do resultado
df_results = get_count_shootouts(df_results)
df_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,datetime,...,away_ratio_win,away_ratio_win_fifa,away_median_score,away_median_fifa_score,summary_ratio_win,summary_ratio_win_fifa,summary_median_score,summary_median_fifa_score,home_ratio_win_shootouts,away_ratio_win_shootouts
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1872-11-30,...,0.511390,0.415385,1.0,1.0,0.333333,0.0,1.0,0.0,1.000000,0.272727
1,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,1874-03-07,...,0.511390,0.415385,1.0,1.0,0.333333,0.0,1.0,0.0,1.000000,0.272727
2,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1875-03-06,...,0.443515,0.150000,1.0,1.0,0.333333,0.0,1.0,0.0,0.272727,1.000000
3,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,1876-03-04,...,0.511390,0.415385,1.0,1.0,0.333333,0.0,1.0,0.0,1.000000,0.272727
4,1877-03-03,England,Scotland,1.0,3.0,Friendly,London,England,False,1877-03-03,...,0.443515,0.150000,1.0,1.0,0.333333,0.0,1.0,0.0,0.272727,1.000000


### Sumarização do resultados de jogos por seleções (historical_win-loose-draw_ratios.csv)

Listagem da relação entre times com a contagem dos jogos, porcentagem de vitórias, porcentagem de derrotas e porcentagem de empates

In [ ]:
# Leitura da "Sumarização do resultados de jogos por seleções"
df_ratios = pd.read_csv(path_source + 'historical_win-loose-draw_ratios.csv')

# Visualização dos dados importados
df_ratios.head()

,country1,country2,games,wins,looses,draws
0,Argentina,Australia,7,0.714286,0.142857,0.142857
1,Australia,Argentina,7,0.142857,0.714286,0.142857
2,Argentina,Belgium,4,0.750000,0.250000,0.000000
3,Belgium,Argentina,4,0.250000,0.750000,0.000000
4,Argentina,Brazil,108,0.361111,0.398148,0.240741


In [ ]:
df_ratios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   country1  798 non-null    object 
 1   country2  798 non-null    object 
 2   games     798 non-null    int64  
 3   wins      798 non-null    float64
 4   looses    798 non-null    float64
 5   draws     798 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 37.5+ KB


Geração das lista com os nomes dos países diferentes usados para comparação com os resultados


In [ ]:
ratios_countries = df_ratios.country1.to_list() + df_ratios.country2.to_list()
df_ratios_countries = pd.DataFrame(ratios_countries, columns = ['country'])
ratios_countries = df_ratios_countries.sort_values('country').country.unique()
ratios_countries

array(['Argentina', 'Australia', 'Belgium', 'Brazil', 'Cameroon',
       'Canada', 'Costa Rica', 'Croatia', 'Denmark', 'Ecuador', 'England',
       'France', 'Germany', 'Ghana', 'Iran', 'Japan', 'Mexico', 'Morocco',
       'Netherlands', 'Poland', 'Portugal', 'Qatar', 'Saudi Arabia',
       'Senegal', 'Serbia', 'South Korea', 'Spain', 'Switzerland',
       'Tunisia', 'Uruguay', 'Wales'], dtype=object)

Comparação da lista do sumário com a fonte de dados dos resultados, para validar se a lista é a mesma

In [ ]:
df_results_countries[df_results_countries.country.isin(ratios_countries)].country.unique()

array(['Argentina', 'Australia', 'Belgium', 'Brazil', 'Cameroon',
       'Canada', 'Costa Rica', 'Croatia', 'Denmark', 'Ecuador', 'England',
       'France', 'Germany', 'Ghana', 'Iran', 'Japan', 'Mexico', 'Morocco',
       'Netherlands', 'Poland', 'Portugal', 'Qatar', 'Saudi Arabia',
       'Senegal', 'Serbia', 'South Korea', 'Spain', 'Switzerland',
       'Tunisia', 'Uruguay', 'Wales'], dtype=object)

#### Organização dos campos

In [ ]:
# Renomear as colunas para unir com resultados
df_ratios = df_ratios.rename(columns={
    'country1': 'home_team',
    'country2': 'away_team',
    'games': 'home_games',
    'wins': 'home_wins',
    'looses': 'home_looses',
    'draws': 'home_draws'
})

# Replicar para os times visitantes
df_ratios['away_games'] = df_ratios.home_games
df_ratios['away_wins'] = df_ratios.home_wins
df_ratios['away_looses'] = df_ratios.home_looses
df_ratios['away_draws'] = df_ratios.home_draws

In [ ]:
# Definição de campos ratio
home_fields_ratios = ['home_games', 'home_wins', 'home_looses', 'home_draws']
away_fields_ratios = ['away_games', 'away_wins', 'away_looses', 'away_draws']

# Remover os outliers da home
for col_name in home_fields_ratios:
  df_ratios = remove_outlier(df_ratios, col_name)

Campo home_games limites -10.000 a 22.000 contagem 94
Campo home_wins limites -1.000 a 1.667 contagem 0
Campo home_looses limites -1.000 a 1.667 contagem 0
Campo home_draws limites -0.547 a 0.912 contagem 36


#### Unificação dos dados da sumarização 

In [ ]:
# Mediana dos valores por paises da sumárização
median_by_country = df_ratios.groupby('home_team').median()

# Mediana dos valores por paises da sumárização
median_general = df_ratios.median(numeric_only=True)

def get_ratios(datasource):
  # Unir o "Histórico de resultados" com 
  # "Sumarização do resultados de jogos por seleções"
  result = pd.merge(datasource, df_ratios, how='left', left_on=['home_team', 'away_team'], right_on=['home_team', 'away_team'])
  
  # Completar os campos vazios com a mediana do país com os outros times
  for index, row in median_by_country.iterrows():
    for field in home_fields_ratios:
      result.loc[(result.home_team == index) & result[field].isna(), [field]] = row[field]
    for field in away_fields_ratios:
      result.loc[(result.away_team == index) & result[field].isna(), [field]] = row[field]

  # Completar os campos vazios 
  # com a mediana do país com os outros times
  for field in home_fields_ratios:
    result.loc[result[field].isna(), [field]] = median_general[field]
  for field in away_fields_ratios:
    result.loc[result[field].isna(), [field]] = median_general[field]    

  return result

df_results = get_ratios(df_results)
df_results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,datetime,...,home_ratio_win_shootouts,away_ratio_win_shootouts,home_games,home_wins,home_looses,home_draws,away_games,away_wins,away_looses,away_draws
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1872-11-30,...,1.000000,0.272727,4.0,0.333333,0.333333,0.166667,5.5,0.581169,0.023810,0.309524
1,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,1874-03-07,...,1.000000,0.272727,4.0,0.333333,0.333333,0.166667,5.5,0.581169,0.023810,0.309524
2,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1875-03-06,...,0.272727,1.000000,5.5,0.581169,0.023810,0.309524,4.0,0.333333,0.333333,0.166667
3,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,1876-03-04,...,1.000000,0.272727,4.0,0.333333,0.333333,0.166667,5.5,0.581169,0.023810,0.309524
4,1877-03-03,England,Scotland,1.0,3.0,Friendly,London,England,False,1877-03-03,...,0.272727,1.000000,5.5,0.581169,0.023810,0.309524,4.0,0.333333,0.333333,0.166667


### Classificação das seleções (ranking.csv)




In [ ]:
# Leitura da "Classificação das seleções por períodos"
df_ranking = pd.read_csv(path_source + 'ranking.csv')

# Apresentação dos dados da "Classificação das seleções por períodos"
df_ranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [ ]:
df_ranking.nunique()

rank                211
country_full        224
country_abrv        217
total_points       3181
previous_points    3030
rank_change         122
confederation         6
rank_date           322
dtype: int64

In [ ]:
ranking_countries = np.array(df_ranking.sort_values('country_full').country_full.unique()).tolist()

print("Valores em resultado e não no ranking")
print(df_results_countries[~df_results_countries.country.isin(ranking_countries)].country.unique())

print("\nValores em ranking e não no resultado")
print(df_ranking[~df_ranking.country_full.isin(results_countries)].sort_values('country_full').country_full.unique())

Valores em resultado e não no ranking
['Abkhazia' 'Alderney' 'Andalusia' 'Arameans Suryoye' 'Artsakh' 'Asturias'
 'Aymara' 'Barawa' 'Basque Country' 'Biafra' 'Bonaire' 'Brittany' 'Brunei'
 'Canary Islands' 'Cape Verde' 'Cascadia' 'Catalonia' 'Central Spain'
 'Chagos Islands' 'Chameria' 'Corsica' 'County of Nice' 'Crimea'
 'DR Congo' 'Ellan Vannin' 'Falkland Islands' 'Felvidék' 'French Guiana'
 'Frøya' 'Galicia' 'German DR' 'Gotland' 'Gozo' 'Greenland' 'Guadeloupe'
 'Guernsey' 'Găgăuzia' 'Hitra' 'Iran' 'Iraqi Kurdistan' 'Isle of Man'
 'Isle of Wight' 'Ivory Coast' 'Jersey' 'Kabylia' 'Kernow' 'Kiribati'
 'Kyrgyzstan' 'Kárpátalja' 'Madrid' 'Manchukuo' 'Mapuche' 'Martinique'
 'Matabeleland' 'Maule Sur' 'Mayotte' 'Menorca' 'Micronesia' 'Monaco'
 'North Korea' 'North Vietnam' 'Northern Cyprus'
 'Northern Mariana Islands' 'Occitania' 'Orkney' 'Padania' 'Panjab'
 'Parishes of Jersey' 'Provence' 'Raetia' 'Republic of St. Pauli' 'Rhodes'
 'Romani people' 'Réunion' 'Saare County' 'Saarland' 'Sain

In [ ]:
# Relação dos nomes dos países
name_countries_by_usa = {
      "Cabo Verde": "Cape Verde",
      "Cape Verde Islands": "Cape Verde",
      'Congo DR': 'DR Congo',
      "Curacao": "Curaçao",
      "Netherlands Antilles": "Curaçao",
      "German DR": "Germany", 
      'USA': 'United States',
      'Korea DPR': 'North Korea',
      'Korea Republic': 'South Korea',
      'IR Iran': 'Iran',
      "Swaziland": "Eswatini",
      'Chinese Taipei': 'Taiwan',
      "Côte d'Ivoire": 'Ivory Coast',
      "US Virgin Islands": "United States Virgin Islands",
      "Saint Vincent and the Grenadines": "Saint Vincent and the Grenadines",
      "St. Vincent and the Grenadines": "Saint Vincent and the Grenadines",
      "St. Vincent / Grenadines": "Saint Vincent and the Grenadines",
      "St. Kitts and Nevis": "Saint Kitts and Nevis",
      "St. Lucia": "Saint Lucia",
      "São Tomé e Príncipe": "São Tomé and Príncipe",
      "Sao Tome e Principe": "São Tomé and Príncipe",
      "FYR Macedonia": "North Macedonia",
      "Türkiye": "Turkey",
      "Kyrgyz Republic": "Kyrgyzstan"
    }

# Renomear os valores na coluna 
df_ranking.country_full = df_ranking.country_full.replace(name_countries_by_usa)

# Apresentar os nomes nos ranking sem relação com a lista de resultados
df_ranking[~df_ranking.country_full.isin(results_countries)].sort_values('country_full').country_full.unique()

array(['Serbia and Montenegro', 'Zaire'], dtype=object)

In [ ]:
df_ranking[df_ranking.country_full == "Germany"].head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
243,1,Germany,GER,58.0,57.0,0,UEFA,1993-08-08
401,5,Germany,GER,56.0,58.0,4,UEFA,1993-09-23
509,4,Germany,GER,58.0,56.0,-1,UEFA,1993-10-22
699,3,Germany,GER,59.0,58.0,-1,UEFA,1993-11-19


In [ ]:
# Conversão da data do rank em formato de data
df_ranking['datetime'] = pd.to_datetime(df_ranking.rank_date, format='%Y-%m-%d')
# Obter os anos de ranking
df_ranking['year'] = pd.DatetimeIndex(df_ranking.datetime).year

# Buscar a última pontuação 
last_ranking = df_ranking.sort_values('year', ascending=False).groupby('country_full').head(1)

# Visualizar resumo dos rankings
last_ranking.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 63915 to 396
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   rank             215 non-null    int64         
 1   country_full     215 non-null    object        
 2   country_abrv     215 non-null    object        
 3   total_points     215 non-null    float64       
 4   previous_points  215 non-null    float64       
 5   rank_change      215 non-null    int64         
 6   confederation    215 non-null    object        
 7   rank_date        215 non-null    object        
 8   datetime         215 non-null    datetime64[ns]
 9   year             215 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 18.5+ KB


In [ ]:
# Renomear o resultado para o time da casa
home_last_ranking = last_ranking[['country_full', 'confederation', 'year', 'rank', 'total_points']].rename(columns={'country_full':'home_team', 'confederation': 'home_confederation', 'rank':'home_last_rank', 'total_points':'home_last_points'})
# Renomear o resultado para o time de fora
away_last_ranking = last_ranking[['country_full', 'confederation', 'year', 'rank', 'total_points']].rename(columns={'country_full':'away_team', 'confederation': 'away_confederation', 'rank':'away_last_rank', 'total_points':'away_last_points'})

def get_ranking(datasource):
  # Unir com última pontuação do time da casa
  result = pd.merge(datasource, home_last_ranking, how='left',
                            left_on=['home_team', 'year'],
                            right_on=['home_team', 'year'])
  # Unir com última pontuação do time de fora
  result = pd.merge(result, away_last_ranking, how='left',
                            left_on=['away_team', 'year'],
                            right_on=['away_team', 'year'])

  # Completar mediana dos últimos ranks quanto nulos
  result.loc[result.home_last_rank.isna(), ['home_last_rank']] = last_ranking['rank'].median()
  result.loc[result.away_last_rank.isna(), ['away_last_rank']] = last_ranking['rank'].median()
  # Completar mediana dos últimos valores quanto nulos
  result.loc[result.home_last_points.isna(), ['home_last_points']] = last_ranking.total_points.median()
  result.loc[result.away_last_points.isna(), ['away_last_points']] = last_ranking.total_points.median()

  return result

# Executar função para apuração do ranking
df_results = get_ranking(df_results)
display(df_results)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,datetime,...,away_games,away_wins,away_looses,away_draws,home_confederation,home_last_rank,home_last_points,away_confederation,away_last_rank,away_last_points
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False,1872-11-30,...,5.5,0.581169,0.023810,0.309524,NaN,104.0,1173.89,NaN,104.0,1173.89
1,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False,1874-03-07,...,5.5,0.581169,0.023810,0.309524,NaN,104.0,1173.89,NaN,104.0,1173.89
2,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False,1875-03-06,...,4.0,0.333333,0.333333,0.166667,NaN,104.0,1173.89,NaN,104.0,1173.89
3,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False,1876-03-04,...,5.5,0.581169,0.023810,0.309524,NaN,104.0,1173.89,NaN,104.0,1173.89
4,1877-03-03,England,Scotland,1.0,3.0,Friendly,London,England,False,1877-03-03,...,4.0,0.333333,0.333333,0.166667,NaN,104.0,1173.89,NaN,104.0,1173.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37945,2022-09-27,Ukraine,Scotland,0.0,0.0,UEFA Nations League,Kraków,Poland,True,2022-09-27,...,4.0,0.333333,0.333333,0.166667,UEFA,27.0,1535.08,UEFA,39.0,1472.66
37946,2022-09-27,Albania,Iceland,1.0,1.0,UEFA Nations League,Tirana,Albania,False,2022-09-27,...,4.0,0.333333,0.333333,0.166667,UEFA,66.0,1371.86,UEFA,63.0,1380.85
37947,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,2022-09-27,...,4.0,0.200000,0.500000,0.166667,UEFA,41.0,1463.50,UEFA,25.0,1547.53
37948,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,2022-09-27,...,4.0,0.333333,0.333333,0.166667,UEFA,19.0,1584.77,UEFA,65.0,1378.23


## Preparação para o Modelo

Remover dados já apurados e que não vão ajudar na apuração pelo contexto da Copa do Mundo, todos os jogos serão no Quatar

In [ ]:
# Remover as colunas usadas para tratamento de dados
df_results = df_results.drop(columns=[
      'date', 
      'datetime', 
      'city', 
      'neutral'
    ])

### Visualização de dados preparados

In [ ]:
# Visualizar os dados alinhados para predição
df_results.head()

,home_team,away_team,home_score,away_score,tournament,country,dayofweek,weekofyear,month,year,...,away_games,away_wins,away_looses,away_draws,home_confederation,home_last_rank,home_last_points,away_confederation,away_last_rank,away_last_points
0,Scotland,England,0.0,0.0,Friendly,Scotland,5,48,11,1872,...,5.5,0.581169,0.023810,0.309524,NaN,104.0,1173.89,NaN,104.0,1173.89
1,Scotland,England,2.0,1.0,Friendly,Scotland,5,10,3,1874,...,5.5,0.581169,0.023810,0.309524,NaN,104.0,1173.89,NaN,104.0,1173.89
2,England,Scotland,2.0,2.0,Friendly,England,5,9,3,1875,...,4.0,0.333333,0.333333,0.166667,NaN,104.0,1173.89,NaN,104.0,1173.89
3,Scotland,England,3.0,0.0,Friendly,Scotland,5,9,3,1876,...,5.5,0.581169,0.023810,0.309524,NaN,104.0,1173.89,NaN,104.0,1173.89
4,England,Scotland,1.0,3.0,Friendly,England,5,9,3,1877,...,4.0,0.333333,0.333333,0.166667,NaN,104.0,1173.89,NaN,104.0,1173.89


In [ ]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37950 entries, 0 to 37949
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   home_team                  37950 non-null  object 
 1   away_team                  37950 non-null  object 
 2   home_score                 37950 non-null  float64
 3   away_score                 37950 non-null  float64
 4   tournament                 37950 non-null  object 
 5   country                    37950 non-null  object 
 6   dayofweek                  37950 non-null  object 
 7   weekofyear                 37950 non-null  UInt32 
 8   month                      37950 non-null  int64  
 9   year                       37950 non-null  int64  
 10  home_ratio_win             37950 non-null  float64
 11  home_ratio_win_fifa        37950 non-null  float64
 12  home_median_score          37950 non-null  float64
 13  home_median_fifa_score     37950 non-null  flo

### Salvar arquivos preparados

In [ ]:
# Salvar dados tratados para arquivo
df_results.to_csv('historical_ready.csv')

#from google.colab import files
#files.download('historical_ready.csv')

---

## Aplicação do tratamento para as partidas agendadas

In [ ]:
df_schedule = pd.read_csv(path_source + 'matches-schedule.csv')

# Renomear colunas para padronização dos campos
df_schedule = df_schedule.rename(columns={'country1': 'home_team', 'country2': 'away_team'})

# Converter texto para date em pandas
df_schedule['datetime'] = pd.to_datetime(df_schedule.date, format='%d/%m/%Y')

# Adicionar dados do torneio e país para todos os jogos agendados
df_schedule['tournament'] = 'FIFA World Cup'
df_schedule['country'] = 'Qatar'

# Renomear os valores na coluna para predição
df_schedule.home_team = df_schedule.home_team.replace({'USA': 'United States'})
df_schedule.away_team = df_schedule.away_team.replace({'USA': 'United States'})

# Aplicação das funções de enriquecimento
get_dates(df_schedule)
df_schedule = get_country_summary(df_schedule)
df_schedule = get_ratios(df_schedule)
df_schedule = get_count_shootouts(df_schedule)
df_schedule = get_ranking(df_schedule)

# Remover campos de orientação
df_schedule = df_schedule.drop(columns=['match', 'phase', 'date', 'datetime'])

# Tem nulo?
print(df_schedule.isnull().values.any())

# Salvar dados tratados para arquivo
df_schedule.to_csv('schedule_ready.csv')

#from google.colab import files
#files.download('schedule_ready.csv')

df_schedule.head()

False


,home_team,away_team,tournament,country,dayofweek,weekofyear,month,year,home_ratio_win,home_ratio_win_fifa,...,away_looses,away_draws,home_ratio_win_shootouts,away_ratio_win_shootouts,home_confederation,home_last_rank,home_last_points,away_confederation,away_last_rank,away_last_points
0,Qatar,Ecuador,FIFA World Cup,Qatar,0,47,11,2022,0.394477,0.000000,...,0.333333,0.333333,0.500000,0.666667,AFC,51,1441.41,CONMEBOL,46,1452.63
1,Senegal,Netherlands,FIFA World Cup,Qatar,0,47,11,2022,0.441989,0.375000,...,0.055556,0.267857,0.529412,0.222222,CAF,20,1584.16,UEFA,10,1658.66
2,England,Iran,FIFA World Cup,Qatar,0,47,11,2022,0.511390,0.415385,...,0.500000,0.090909,0.272727,0.388889,UEFA,5,1761.71,AFC,21,1564.49
3,United States,Wales,FIFA World Cup,Qatar,0,47,11,2022,0.418530,0.266667,...,0.529412,0.100000,0.571429,0.000000,CONCACAF,15,1633.72,UEFA,18,1588.08
4,France,Australia,FIFA World Cup,Qatar,1,47,11,2022,0.472409,0.436364,...,0.200000,0.200000,0.444444,0.625000,UEFA,4,1764.85,AFC,42,1462.29


In [ ]:
df_schedule.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 47
Data columns (total 36 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   home_team                  48 non-null     object 
 1   away_team                  48 non-null     object 
 2   tournament                 48 non-null     object 
 3   country                    48 non-null     object 
 4   dayofweek                  48 non-null     object 
 5   weekofyear                 48 non-null     UInt32 
 6   month                      48 non-null     int64  
 7   year                       48 non-null     int64  
 8   home_ratio_win             48 non-null     float64
 9   home_ratio_win_fifa        48 non-null     float64
 10  home_median_score          48 non-null     float64
 11  home_median_fifa_score     48 non-null     float64
 12  away_ratio_win             48 non-null     float64
 13  away_ratio_win_fifa        48 non-null     float64
 

---

## Geração de pré-processadores

In [ ]:
# Importação de ferramentas 
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# Listagem de campos numéricos
numeric_features = [
      'weekofyear',
      'month',
      'year',

      'home_games', 
      'home_wins', 
      'home_looses', 
      'home_draws',
      'home_ratio_win_shootouts',   
      'home_median_score',
      'home_ratio_win',
      'home_ratio_win_fifa',
      'home_median_fifa_score',      
      'home_last_rank',
      'home_last_points',      

      'away_games', 
      'away_wins', 
      'away_looses', 
      'away_draws',
      'away_ratio_win_shootouts',
      'away_ratio_win',      
      'away_ratio_win_fifa',
      'away_median_score',
      'away_median_fifa_score',
      'away_last_rank',
      'away_last_points',

      'summary_ratio_win',
      'summary_ratio_win_fifa',
      'summary_median_score',
      'summary_median_fifa_score'
    ]
# Transformação para valores numéricos
numeric_transformer = Pipeline(steps=[
      ("imputer", SimpleImputer(strategy="median")), 
      ("scaler", StandardScaler())
    ])

# Listagem de campo categóricos (texto)
categorical_features = [
      'home_team',
      'home_confederation',
      'away_team',
      'away_confederation',
      'tournament',
      'country',
      'dayofweek'
  ]
# Transformação para valores categóricos
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# Geração do preprocessador de transformação
preprocessor = ColumnTransformer(transformers=[
      ("num", numeric_transformer, numeric_features),
      ("cat", categorical_transformer, categorical_features),
    ])

### Segregação das fontes de dados

Primeiro para classificação e atributos

Posteriormente segregação entre base de treino e teste 

In [ ]:
# Biblioteca de Separação de Treino/Teste
from sklearn.model_selection import train_test_split

# Separação de atributos e classficadores
dataset_x = df_results.drop(['home_score', 'away_score'], axis=1)
dataset_y = df_results[['home_score', 'away_score']]

# Separação da amostra de treino e de testes 
train_x, test_x, train_y, test_y = train_test_split(dataset_x, dataset_y, test_size=0.3, random_state=42)
'''
# Critério de Copa do Mundo de 2018
games_fifa_worldcup_2018 = (df_results.year == 2018) & (df_results.tournament == 'FIFA World Cup')

# Separação de atributos com critério de Copa do Mundo de 2018
train_x = df_results[~games_fifa_worldcup_2018].drop(['home_score', 'away_score'], axis=1)
test_x = df_results[games_fifa_worldcup_2018].drop(['home_score', 'away_score'], axis=1)

# Separação de classficadores com critério de Copa do Mundo de 2018
train_y = df_results[~games_fifa_worldcup_2018][['home_score', 'away_score']]
test_y = df_results[games_fifa_worldcup_2018][['home_score', 'away_score']]
'''

"\n# Critério de Copa do Mundo de 2018\ngames_fifa_worldcup_2018 = (df_results.year == 2018) & (df_results.tournament == 'FIFA World Cup')\n\n# Separação de atributos com critério de Copa do Mundo de 2018\ntrain_x = df_results[~games_fifa_worldcup_2018].drop(['home_score', 'away_score'], axis=1)\ntest_x = df_results[games_fifa_worldcup_2018].drop(['home_score', 'away_score'], axis=1)\n\n# Separação de classficadores com critério de Copa do Mundo de 2018\ntrain_y = df_results[~games_fifa_worldcup_2018][['home_score', 'away_score']]\ntest_y = df_results[games_fifa_worldcup_2018][['home_score', 'away_score']]\n"

## Modelagem e Apuração das Predições

In [ ]:
'''
# Importação do modelo para apreendizado
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.tree import DecisionTreeClassifier

# Inicialização do modelo com a parametrização
cdt = DecisionTreeClassifier(random_state=42)
multicdt = MultiOutputClassifier(cdt)

# Geração do pipeline de previsão
classifierDecisionTree = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", multicdt)])

# Treinamento do modelo
classifierDecisionTree.fit(train_x, train_y)

# Pontuação para Decision Tree Classifier
print("Decision Tree Classifier - score: %.3f" % classifierDecisionTree.score(test_x, test_y))
'''

'\n# Importação do modelo para apreendizado\nfrom sklearn.multioutput import MultiOutputClassifier \nfrom sklearn.tree import DecisionTreeClassifier\n\n# Inicialização do modelo com a parametrização\ncdt = DecisionTreeClassifier(random_state=42)\nmulticdt = MultiOutputClassifier(cdt)\n\n# Geração do pipeline de previsão\nclassifierDecisionTree = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", multicdt)])\n\n# Treinamento do modelo\nclassifierDecisionTree.fit(train_x, train_y)\n\n# Pontuação para Decision Tree Classifier\nprint("Decision Tree Classifier - score: %.3f" % classifierDecisionTree.score(test_x, test_y))\n'

In [ ]:
'''
# Importação do modelo para apreendizado e controlador de várias saídas
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.ensemble import RandomForestClassifier

# Inicialização do modelo com a parametrização
rforest = RandomForestClassifier(random_state=42)
multiRforest = MultiOutputClassifier(rforest)

# Geração do pipeline de previsão
classifierForest = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", multiRforest)])

# Treinamento do modelo
classifierForest.fit(train_x, train_y)

# Pontuação para Random Forest
print("Random Forest - score: %.3f" % classifierForest.score(test_x, test_y))
'''

'\n# Importação do modelo para apreendizado e controlador de várias saídas\nfrom sklearn.multioutput import MultiOutputClassifier \nfrom sklearn.ensemble import RandomForestClassifier\n\n# Inicialização do modelo com a parametrização\nrforest = RandomForestClassifier(random_state=42)\nmultiRforest = MultiOutputClassifier(rforest)\n\n# Geração do pipeline de previsão\nclassifierForest = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", multiRforest)])\n\n# Treinamento do modelo\nclassifierForest.fit(train_x, train_y)\n\n# Pontuação para Random Forest\nprint("Random Forest - score: %.3f" % classifierForest.score(test_x, test_y))\n'

In [ ]:
# Importação do modelo para apreendizado e controlador de várias saídas
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.ensemble import GradientBoostingClassifier

# Inicialização do modelo com a parametrização
gbc = GradientBoostingClassifier(random_state=42)
multiOutputGbc = MultiOutputClassifier(gbc)

# Geração do Pipeline de previsão
classifierGBC = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", multiOutputGbc)])

# Predição e pontuação para Gradient Boosting Classifier 
classifierGBC.fit(train_x, train_y)
print("Gradient Boosting Classifier - score: %.3f" % classifierGBC.score(test_x, test_y))

Gradient Boosting Classifier - score: 0.143


### Tunando o Gradient Boosting Classifier

In [ ]:
'''
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.ensemble import GradientBoostingClassifier

param_distributions = {
    "classifier__estimator__n_estimators": [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000],
    "classifier__estimator__max_leaf_nodes": [2, 5, 10, 20, 50, 100],
    "classifier__estimator__learning_rate": [0.01, 0.1, 1],
    "classifier__estimator__max_depth": [3, 4, 5]
  }

# Geração do pipeline 
pipe_gbc = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))
  ])

# Gerar grade para busca dos parâmetros
gs_gbc = GridSearchCV(estimator=pipe_gbc, param_grid=param_distributions)

# Treinar os modelos para busca
gs_gbc.fit(train_x, train_y)

# Visualizar os resultados 
print(gs_gbc.best_estimator_)
'''

'\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.multioutput import MultiOutputClassifier \nfrom sklearn.ensemble import GradientBoostingClassifier\n\nparam_distributions = {\n    "classifier__estimator__n_estimators": [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000],\n    "classifier__estimator__max_leaf_nodes": [2, 5, 10, 20, 50, 100],\n    "classifier__estimator__learning_rate": [0.01, 0.1, 1],\n    "classifier__estimator__max_depth": [3, 4, 5]\n  }\n\n# Geração do pipeline \npipe_gbc = Pipeline(steps=[\n    ("preprocessor", preprocessor),\n    ("classifier", MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))\n  ])\n\n# Gerar grade para busca dos parâmetros\ngs_gbc = GridSearchCV(estimator=pipe_gbc, param_grid=param_distributions)\n\n# Treinar os modelos para busca\ngs_gbc.fit(train_x, train_y)\n\n# Visualizar os resultados \nprint(gs_gbc.best_estimator_)\n'

In [ ]:
# Importação do modelo para apreendizado e controlador de várias saídas
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Inicialização do modelo com a parametrização
gbc = GradientBoostingClassifier(random_state=42)
multiOutputGbc = MultiOutputClassifier(gbc)

# Geração do Pipeline de previsão
classifierGBC = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", multiOutputGbc)])

# Treinar o Gradient Boosting Classifier 
classifierGBC.fit(dataset_x, dataset_y)

# Carregar a fonte de dados preparada
df_schedule_ready = pd.read_csv('schedule_ready.csv')
df_schedule_ready.dayofweek = df_schedule_ready.dayofweek.astype(str)
df_schedule_ready[['home_score', 'away_score']] = classifierGBC.predict(df_schedule_ready)

# Remover e renomear colunas para salvar resultado
df_schedule_ready = df_schedule_ready.drop(columns=numeric_features)
df_schedule_ready = df_schedule_ready.drop(columns=['Unnamed: 0', 'tournament', 'country', 'dayofweek', 'home_confederation', 'away_confederation' ])
df_schedule_ready = df_schedule_ready.rename(columns={'home_team': 'home', 'away_team': 'away'})

# Renomear os valores na coluna para predição
df_schedule_ready.home = df_schedule_ready.home.replace({'United States': 'USA'})
df_schedule_ready.away = df_schedule_ready.away.replace({'United States': 'USA'})

# Geração do resultado da previsão
df_schedule_ready.to_csv('result.csv', index=False)

# Download file with game results
#from google.colab import files
#files.download('result.csv')

## Referências

[Daryl Felix - Football WorldCup Predictive Model](https://resquatordaryl.medium.com/football-worldcup-predictive-model-9a313ebd28b7)

[Prediction of the FIFA World Cup 2018](https://arxiv.org/pdf/1806.03208v3.pdf)

[Nick Hotz - What is CRISP DM?](https://www.datascience-pm.com/crisp-dm-2/)

[Scikit-learn - Multiclass and multioutput algorithms](https://scikit-learn.org/stable/modules/multiclass.html#multiclass-multioutput-classification)